In [49]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, CountVectorizer, RegexTokenizer
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Criar uma SparkSession
spark = SparkSession.builder.appName("").getOrCreate()

# Lista de descrições de filmes
data = [
    (0, "Um jovem descobre que é um mago e vai para uma escola de magia."),
    (1, "Um grupo de amigos embarca em uma jornada para destruir um anel poderoso."),
    (2, '"Several actresses get caught up in a web of romantic intrigue while performing in a production of Shakespeare"s ""Twelfth Night."""')
]

columns = ["id", "description"]
df = spark.createDataFrame(data, columns)

In [51]:
# Tokenizar o texto
regexTokenizer = Tokenizer(inputCol="description", outputCol="words")
wordsData = regexTokenizer.transform(df)
wordsData = wordsData.withColumn("words", F.expr("transform(words, x -> regexp_replace(x, '[\\\\.\\'\\\"]', ''))"))

words_to_remove = ["um", "para", "de", "uma", "e" ,"que", "é", "em", "sua"]
words_to_remove_col = F.array([F.lit(word) for word in words_to_remove])

# Usar array_except para remover as palavras específicas
result_cleaned = wordsData.withColumn("cleaned_words", F.expr("array_except(words, array(" + ", ".join([f"'{word}'" for word in words_to_remove]) + "))"))

# Visualizar o resultado
result_cleaned.show(truncate=False)


+---+------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|id |description                                                                                                                         |words                                                                                                                                            |cleaned_words                                                                                                                         |
+---+------------------------------------------------------------------------------------------------------------------------------------+----------------------

In [32]:
# Supondo que você já tenha um DataFrame result com a coluna "words"
df2 = result.withColumn("words", F.col("words").cast("array<string>"))

# Aplicar a função regexp_replace a cada palavra na lista para remover pontos finais
df2 = df2.withColumn("words", F.expr("transform(words, x -> regexp_replace(x, '\\\.$', ''))"))
df2.select("words").show(truncate=False)

+--------------------------------------------------------------------------------------+
|words                                                                                 |
+--------------------------------------------------------------------------------------+
|[um, jovem, descobre, que, é, um, mago, e, vai, para, uma, escola, de, magia]         |
|[um, grupo, de, amigos, embarca, em, uma, jornada, para, destruir, um, anel, poderoso]|
|[um, super-herói, luta, contra, vilões, para, salvar, sua, cidade]                    |
+--------------------------------------------------------------------------------------+



In [40]:
index = 1

# Selecionar a posição específica do vetor e colocá-la em uma string
df3 = result.withColumn("specific_word", F.expr(f"words[{index}]"))
df3 = df3.withColumn("string_with_word", F.concat(F.lit("A palavra selecionada é:"), F.col("specific_word")))

df3.select("string_with_word").show(truncate=False)

+-----------------------------+
|string_with_word             |
+-----------------------------+
|A palavra selecionada é:jovem|
|A palavra selecionada é:grupo|
|A palavra selecionada é:super|
+-----------------------------+



In [58]:
from deep_translator import GoogleTranslator

def translate_text(text):
    if text is None:
        return None
    try:
        # Traduz para o inglês
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return translated
    except Exception as e:
        return text  # Em caso de erro, retorna o texto original

# Teste da função
print(translate_text("Olá, como vai?"))  # Deve retornar "Hello, how are you?"

Hello, how are you?


In [59]:
translate_udf = F.udf(translate_text, T.StringType())

data = [("Hola, ¿cómo estás?",), ("Bonjour tout le monde",), ("Hello, how are you?",)]
columns = ["overview"]
df = spark.createDataFrame(data, columns)

# Aplicar a função de tradução na coluna 'overview'
df_translated = df.withColumn("overview_translated", translate_udf(df["overview"]))


In [60]:
df_translated.show(truncate=False)

+---------------------+-------------------+
|overview             |overview_translated|
+---------------------+-------------------+
|Hola, ¿cómo estás?   |Hello how are you? |
|Bonjour tout le monde|Hello everyone     |
|Hello, how are you?  |Hello, how are you?|
+---------------------+-------------------+

